# Projeto 2 - Ciência dos Dados

Nome: Amanda Ades

Nome: Beni Kracochansky

Nome: Gabriel Kabbani

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [30]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [31]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [32]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Tesla'

#aumentamos a quantidade de mensagens devido a quantidade de retweets.
#Quantidade mínima de mensagens capturadas:
n = 950
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [ ]:
tweets= pd.ExcelFile('Tesla.xlsx')
treino= pd.read_excel(tweets, sheet_name='Treinamento')
teste= pd.read_excel(tweets, sheet_name='Teste')

#FAZER O GITIGNORE DO AUTH PASS

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [47]:
#Limpando a base de dados:

def limpador(tweet):
    retirar=[',',';','/n','...','***','.','-','|','?','!',':'] #verificar se faltam outros.
    tweet=str(tweet)
    tweet.lower()
    tweet.split()
    palavras=[]
    for palavra in tweet:
        if len(palavra)>0:
            for item in retirar:
                palavra.replace(item," ")
        lista=['@','https://']
        if palavra[0]==lista[0]:
            pass
        if len(palavra)>7:
            if palavra[:8]==lista[2]:
                pass
        else:
            palavras.append(palavra)
    return palavras

tweets=[]
for tweet in treino['Treinamento']: #limpando todos os tweets da sheet 'Treinamento', e adicionando eles a uma lista.
        tweets.append(" ".join(limpador(tweet)))
for e in range (len(treino['Treinamento'])): #limpa o dataframe com os dados obtidos acima.
    if tweets[e][:1] != 'rt': #removendo retweets
        treino['Treinamento'][e]=tweets[e]
        
treino['Treinamento'][3] #não sei por que ele nao consegue obter esse valor, parece que ta num loop infinito.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'r                                                                                                                                                                                                                                                               t                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               @                                                                                                                                                                                                                                      

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
#tirar os retweets denovo, mas p/ testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**